In [25]:
#Update your token
STUDENT_TOKEN = 'MAUDANDJOSPEH'

Server communication game and game_loop.

In [26]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Game specific code

In [27]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Domineering(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    actions = []
    board = self.get_board()
    player = self.player

    for row in range(len(board)):
      for col in range(len(board[row])):
        if board[row][col] == '.':
          if player == 'H' and col + 1 < len(board[row]) and board[row][col + 1] == '.':
            actions.append((row, col))
          elif player == 'V' and row + 1 < len(board) and board[row + 1][col] == '.':
            actions.append((row, col))
    return actions

  
  def print_game(self):
    for row in self.get_board():
      print(row)

  def other_player(self):
    if self.player == 'V': return 'H'
    if self.player == 'H': return 'V'
    


In [28]:
#Helper functions
def check_if_in(array1, array2):
  for element in array1:
    if element in array2:
      return True, list(set(array1) & set(array2))
  return False, None

def print_game(board):
  for row in board:
    print(row)

def actions(board, player):
  actions = []

  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == '.':
        if player == 'H' and col + 1 < len(board[row]) - 1 and board[row][col + 1] == '.':
          actions.append((row, col))
        elif player == 'V' and row + 1 < len(board) and board[row + 1][col] == '.':
          actions.append((row, col))
  return actions

def configure_board(game, action, player):
  row, col = action
  temp_board = deepcopy(game.get_board())
  temp_board[row][col] = player
  #add the move/action we are calculating for
  if player == 'V':
    temp_board[row + 1][col] = player
  elif player == 'H':
    temp_board[row][col + 1] = player
  return temp_board

In [29]:
#other functions
def moves_opponent(game, action, player):
  temp_board = configure_board(game, action, player)
  print("Temp board")
  print_game(temp_board)
  player_moves = len(actions(temp_board, player))
  return player_moves
  #print(f"Opponent moves: {opponent_moves}, Player moves: {player_moves}")
  
  
def make_dict(game, actions):
  possible_moves = {}
  for action in actions:
    number = moves_opponent(game, action, game.player)
    if number not in possible_moves.keys():
      possible_moves[number] = [action]
    else:
      possible_moves[number].append(action)

  return possible_moves

def moves_player(game, board, player_moves):
  opponent_moves = {}
  player = game.other_player()
  max_key = -1000
  for move in player_moves:
    board = configure_board(game, move, player)
    print(f"Board with possible player movement in {move}")
    possible_actions = actions(board, player)
    for action in possible_actions:
      number = moves_opponent(game, action, player)
      if number not in opponent_moves.keys():
        opponent_moves[number] = [action]
      else:
        opponent_moves[number].append(action)
    #find min key of opponent moves
    temp_key = min(opponent_moves.keys())
    if temp_key > max_key:
      max_key = temp_key
      print("New max key for US after opponent move: ", max_key)
  
  return max_key



In [30]:
#solver
def solver(game):
    preferred_orientation_H = [(0, 1), (0, 3), (5, 1), (5, 3)]
    preferred_orientation_V = [(1, 0), (3, 0), (1, 5), (3, 5)] 
    actions = game.actions()
    
    if game.player == 'H':
      bool, common_coordinates = check_if_in(preferred_orientation_H, actions)
      if bool:
        choice = random.choice(common_coordinates)
        #print(choice)
        #print("Checkpoint: took from preferred orientation")
        return choice
      
    elif game.player == 'V':
      bool, common_coordinates = check_if_in(preferred_orientation_V, actions)
      if bool:
        choice = random.choice(common_coordinates)
        #print(choice)
        #print("Checkpoint: took from preferred orientation")
        return choice
    
  
    possible_moves = make_dict(game, actions)
    player_dict = dict(sorted(possible_moves.items(), key=lambda item: item[0], reverse=True))
    max_key = max(player_dict.keys())
    result_list = player_dict[max_key]
    print("Best moves for us to make: ", result_list)
    
    int_moves_we_can_make = moves_player(game, game.get_board(), result_list) #return the maximum moves we can make
    print("int_moves_we_can_make: ", int_moves_we_can_make)
    
    
    choice = random.choice(result_list)
    return choice



In [31]:
game_loop(solver, Domineering, 'domineering', multi_player=False, id=None)

Creating new game...
Joining game with id: 9291
Playing as V
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', 'H', 'H', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', 'H', 'H', '.', '.', '.']
['V', '.', 'H', 'H', '.', 'V']
['V', '.', '.', '.', '.', 'V']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['.', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['V', 'H', 'H', '.', '.', '.']
['V', '.', 'H', 'H', '.', 'V']
['V', '.', 'H', 'H', '.', 'V']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['.', '.', '.', '.'

Game run loop

In [32]:
def run_games(solver, GameClass, game_type, num_games=50):
    results = {"wins": [0, {'H': 0, 'V': 0}], "losses": [0, {'H': 0, 'V': 0}]}
    for _ in range(num_games):
        # Initialize a game
        game_id = new_game(game_type)
        player = join_game(game_type, game_id)

        # Run the game loop
        while True:
            game = game_state(game_type, game_id, GameClass)
            if game is None:
                print("Failed to fetch the game state.")
                break  # Exit the loop if the game state couldn't be fetched

            #game.print_game()
            if game.is_end():
                if game.player == '-':
                    results["draws"][0] += 1
                    results["draws"][1].append(None)
                elif game.player == player:
                    #print("I won!")
                    results["wins"][0] += 1
                    results["wins"][1][player] += 1
                else:
                    #print("I lost!")
                    results["losses"][0] += 1
                    results["losses"][1][player] += 1
                break

            if game.player == player:
                move = solver(game)
                update_game(game_type, game_id, player, json.dumps(move))
            else:
                time.sleep(2)

    return results

In [33]:
results = run_games(solver, Domineering, 'domineering', num_games=10)

Temp board
['.', 'H', 'H', 'H', 'H', 'V']
['V', 'H', 'H', '.', '.', 'V']
['V', '.', '.', '.', 'V', '.']
['.', '.', '.', '.', 'V', '.']
['.', '.', '.', 'V', '.', '.']
['.', 'H', 'H', 'V', '.', '.']
Temp board
['.', 'H', 'H', 'H', 'H', 'V']
['V', '.', 'H', 'H', '.', 'V']
['V', '.', '.', '.', 'V', '.']
['.', '.', '.', '.', 'V', '.']
['.', '.', '.', 'V', '.', '.']
['.', 'H', 'H', 'V', '.', '.']
Temp board
['.', 'H', 'H', 'H', 'H', 'V']
['V', '.', '.', 'H', 'H', 'V']
['V', '.', '.', '.', 'V', '.']
['.', '.', '.', '.', 'V', '.']
['.', '.', '.', 'V', '.', '.']
['.', 'H', 'H', 'V', '.', '.']
Temp board
['.', 'H', 'H', 'H', 'H', 'V']
['V', '.', '.', '.', '.', 'V']
['V', 'H', 'H', '.', 'V', '.']
['.', '.', '.', '.', 'V', '.']
['.', '.', '.', 'V', '.', '.']
['.', 'H', 'H', 'V', '.', '.']
Temp board
['.', 'H', 'H', 'H', 'H', 'V']
['V', '.', '.', '.', '.', 'V']
['V', '.', 'H', 'H', 'V', '.']
['.', '.', '.', '.', 'V', '.']
['.', '.', '.', 'V', '.', '.']
['.', 'H', 'H', 'V', '.', '.']
Temp board
['.'

IndexError: list index out of range

In [ ]:
#writw this to a file called results.txt in the same directory
with open('results.txt', 'a') as f:
    f.write('doing -1 no matter what' + '\n' + json.dumps(results, indent=4) + '\n')

In [ ]:
print(results)

{'wins': [10, {'H': 5, 'V': 5}], 'losses': [0, {'H': 0, 'V': 0}]}
